# Machine Learning 
## Random Forest - Regresion Logistica - Analisis Discriminante - K vecinos - Arboles de Decision - SVM - Gaussian NB

In [2]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
# pymongo
import pymongo
print('pymongo: {}'.format(pymongo.__version__))

Python: 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
scipy: 1.0.1
numpy: 1.14.2
matplotlib: 2.2.2
pandas: 0.23.0
sklearn: 0.19.1
pymongo: 3.4.0


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from pymongo import MongoClient

# Cargamos los datos

### Usando MongoDB

In [55]:
client = MongoClient('localhost', 27017)
db = client.datascience
collection = db.incidents

#### - Usamos un cursor para construir el Dataframe

In [27]:
cursor = collection.find()
incidents = pd.DataFrame(list(cursor))

#### - Borramos el _id que Mongo genera automáticamente


In [28]:
 if no_id:
     del incidents['_id']

 return incidents

### Usando un csv

In [84]:
names = ['category', 'type', 'dayofweek', 'date', 'time','day','year','month','hour','district', 'resolution', 'latitude', 'longitude']
incidents = pd.read_csv("Incidents.all.ml.csv", names=names, sep=';')

### Cargaremos un csv de 2017-18 (menos datos)

In [5]:
names = ['category', 'type', 'dayofweek', 'date', 'time','day','year','month','hour','district', 'resolution', 'latitude', 'longitude']
incidents = pd.read_csv("Incidents.red.ml.csv", names=names, sep=';')

## shape. Dimensiones del conjunto de datos

In [6]:
print(incidents.shape)

(915339, 13)


## head

In [7]:
incidents.head(10)

,category,type,dayofweek,date,time,day,year,month,hour,district,resolution,latitude,longitude
0,VANDALISM,MALICIOUS MISCHIEF,Monday,2007-10-22,14:30:00,22,2007,10,14,TARAVAL,NONE,-122.466758005159,37.729185
1,OTHER OFFENSES,RESISTING ARREST,Sunday,2008-03-02,21:31:00,02,2008,3,21,TENDERLOIN,"ARREST, BOOKED",-122.412224164736,37.782073
2,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Friday,2009-10-30,19:30:00,30,2009,10,19,NORTHERN,NONE,-122.422253466945,37.790863
3,OTHER OFFENSES,PROBATION VIOLATION,Wednesday,2011-01-19,16:25:00,19,2011,1,16,INGLESIDE,"ARREST, BOOKED",-122.447125045686,37.721031
4,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Friday,2008-09-12,13:21:00,12,2008,9,13,MISSION,PSYCHOPATHIC CASE,-122.416954468807,37.753880
5,NON-CRIMINAL,LOST PROPERTY,Thursday,2003-10-02,21:00:00,02,2003,10,21,RICHMOND,NONE,-122.472843756561,37.780629
6,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Monday,2005-02-07,18:50:00,07,2005,2,18,MISSION,"ARREST, BOOKED",-122.418596727467,37.753838
7,VEHICLE THEFT,STOLEN AND RECOVERED VEHICLE,Tuesday,2006-08-22,18:30:00,22,2006,8,18,CENTRAL,NONE,-122.400439442308,37.795223
8,VEHICLE THEFT,STOLEN AUTOMOBILE,Thursday,2012-10-04,17:00:00,04,2012,10,17,INGLESIDE,NONE,-122.405527956606,37.716948
9,OTHER OFFENSES,OBSCENE PHONE CALLS(S),Wednesday,2010-11-17,18:30:00,17,2010,11,18,RICHMOND,NONE,-122.459722209477,37.778342


## Statistical summary. descriptions

In [8]:
incidents.describe()

,year,month,hour,longitude
count,915339.000000,915339.000000,915339.000000,915339.000000
mean,2001.726924,16.591035,13.410482,36.959908
std,141.713059,142.234680,6.518946,11.370903
min,1.000000,1.000000,0.000000,-122.513642
25%,2007.000000,3.000000,9.000000,37.753565
50%,2013.000000,6.000000,14.000000,37.775421
75%,2016.000000,10.000000,19.000000,37.784666
max,2018.000000,2018.000000,23.000000,90.000000


## Class Distribution

In [9]:
incidents.groupby('category').size()

category
ARSON                            1656
ASSAULT                         79695
BAD CHECKS                        352
BRIBERY                           350
BURGLARY                        36699
DISORDERLY CONDUCT               3876
DRIVING UNDER THE INFLUENCE      2221
DRUG/NARCOTIC                   42692
DRUNKENNESS                      3502
EMBEZZLEMENT                     1149
EXTORTION                         335
FAMILY OFFENSES                   415
FORGERY/COUNTERFEITING           8105
FRAUD                           16474
GAMBLING                          146
KIDNAPPING                       1881
LARCENY/THEFT                  223943
LIQUOR LAWS                      1466
LOITERING                         846
MISSING PERSON                  26037
NON-CRIMINAL                    98777
OTHER OFFENSES                 122214
PORNOGRAPHY/OBSCENE MAT            26
PROSTITUTION                     5961
RECOVERED VEHICLE                3729
ROBBERY                         21983
RUN

* Descarte de las columnas para realizar una mejor discretación de los atributos para realizar la clasificación

In [10]:
dataset = incidents.copy()
dataset.drop(dataset.columns[[1,3,4,5,6,7]], axis=1, inplace=True)

In [11]:
incidents.head(5)

,category,type,dayofweek,date,time,day,year,month,hour,district,resolution,latitude,longitude
0,VANDALISM,MALICIOUS MISCHIEF,Monday,2007-10-22,14:30:00,22,2007,10,14,TARAVAL,NONE,-122.466758005159,37.729185
1,OTHER OFFENSES,RESISTING ARREST,Sunday,2008-03-02,21:31:00,02,2008,3,21,TENDERLOIN,"ARREST, BOOKED",-122.412224164736,37.782073
2,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Friday,2009-10-30,19:30:00,30,2009,10,19,NORTHERN,NONE,-122.422253466945,37.790863
3,OTHER OFFENSES,PROBATION VIOLATION,Wednesday,2011-01-19,16:25:00,19,2011,1,16,INGLESIDE,"ARREST, BOOKED",-122.447125045686,37.721031
4,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Friday,2008-09-12,13:21:00,12,2008,9,13,MISSION,PSYCHOPATHIC CASE,-122.416954468807,37.753880


In [12]:
dataset.head(5)

,category,dayofweek,hour,district,resolution,latitude,longitude
0,VANDALISM,Monday,14,TARAVAL,NONE,-122.466758005159,37.729185
1,OTHER OFFENSES,Sunday,21,TENDERLOIN,"ARREST, BOOKED",-122.412224164736,37.782073
2,VANDALISM,Friday,19,NORTHERN,NONE,-122.422253466945,37.790863
3,OTHER OFFENSES,Wednesday,16,INGLESIDE,"ARREST, BOOKED",-122.447125045686,37.721031
4,NON-CRIMINAL,Friday,13,MISSION,PSYCHOPATHIC CASE,-122.416954468807,37.753880


## Normalización de datos

In [14]:
ndataset = dataset.copy()
ndataset.district = pd.Categorical(ndataset.district)
ndataset.dayofweek = pd.Categorical(ndataset.dayofweek)
ndataset.resolution = pd.Categorical(ndataset.resolution)
ndataset.category = pd.Categorical(ndataset.category)

ndataset['district'] = ndataset.district.cat.codes
ndataset['dayofweek'] = ndataset.dayofweek.cat.codes
ndataset['resolution'] = ndataset.resolution.cat.codes
ndataset['category'] = ndataset.category.cat.codes

ndataset.head(5)

,category,dayofweek,hour,district,resolution,latitude,longitude
0,35,4,14,32,15,-122.466758005159,37.729185
1,21,8,21,33,0,-122.412224164736,37.782073
2,35,2,19,28,15,-122.422253466945,37.790863
3,21,11,16,26,0,-122.447125045686,37.721031
4,20,2,13,27,21,-122.416954468807,37.753880


In [15]:
ndataset = ndataset.convert_objects(convert_numeric=True).dropna()
ndataset

,category,dayofweek,hour,district,resolution,latitude,longitude
0,35,4,14,32,15,-122.466758,37.729185
1,21,8,21,33,0,-122.412224,37.782073
2,35,2,19,28,15,-122.422253,37.790863
3,21,11,16,26,0,-122.447125,37.721031
4,20,2,13,27,21,-122.416954,37.753880
5,20,9,21,30,15,-122.472844,37.780629
6,16,4,18,27,0,-122.418597,37.753838
7,36,10,18,25,15,-122.400439,37.795223
8,36,9,17,26,15,-122.405528,37.716948
9,21,11,18,30,15,-122.459722,37.778342


## Dividimos nuestro conjunto de datos en train y test (80-20)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [17]:
x = ndataset.drop('category',axis=1)
y = ndataset['category']

In [18]:
from sklearn import preprocessing
normalized_x = preprocessing.normalize(x)

In [19]:
from sklearn.model_selection import train_test_split
x_training, x_test, y_training, y_test = train_test_split(x, y, test_size=0.2)

In [20]:
scaler.fit(x_training)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [21]:
x_training.shape

(728566, 6)

In [22]:
y_training.shape

(728566,)

In [23]:
x_test.shape

(182142, 6)

In [24]:
y_test.shape

(182142,)

In [25]:
x_training.head(5)

,dayofweek,hour,district,resolution,latitude,longitude
883397,4,15,26,15,-122.440007,37.721805
659255,7,20,32,10,-122.456086,37.716630
135877,4,9,31,1,-122.401817,37.788441
705598,7,11,32,15,-122.481183,37.743727
801842,2,9,30,15,-122.498185,37.780583


In [26]:
x_test.head(5)

,dayofweek,hour,district,resolution,latitude,longitude
365170,9,11,27,15,-122.416726,37.757954
625791,8,18,25,15,-122.412453,37.798066
459490,2,19,31,15,-122.410406,37.778784
225368,7,8,24,15,-122.397121,37.737121
533255,9,9,31,15,-122.403405,37.775421


In [27]:
y_training.head(5)

883397    16
659255    27
135877    34
705598    16
801842    20
Name: category, dtype: int8

In [28]:
y_test.head(5)

365170    16
625791    16
459490    36
225368    20
533255    16
Name: category, dtype: int8

In [29]:
incidents.head(5)

,category,type,dayofweek,date,time,day,year,month,hour,district,resolution,latitude,longitude
0,VANDALISM,MALICIOUS MISCHIEF,Monday,2007-10-22,14:30:00,22,2007,10,14,TARAVAL,NONE,-122.466758005159,37.729185
1,OTHER OFFENSES,RESISTING ARREST,Sunday,2008-03-02,21:31:00,02,2008,3,21,TENDERLOIN,"ARREST, BOOKED",-122.412224164736,37.782073
2,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Friday,2009-10-30,19:30:00,30,2009,10,19,NORTHERN,NONE,-122.422253466945,37.790863
3,OTHER OFFENSES,PROBATION VIOLATION,Wednesday,2011-01-19,16:25:00,19,2011,1,16,INGLESIDE,"ARREST, BOOKED",-122.447125045686,37.721031
4,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Friday,2008-09-12,13:21:00,12,2008,9,13,MISSION,PSYCHOPATHIC CASE,-122.416954468807,37.753880


Aplicamos las transformaciones una vez realizada la normalización de los datos

In [30]:
x_training = scaler.transform(x_training)
x_test = scaler.transform(x_test)

x_training
x_test

array([[ 0.57387139, -0.37478645, -0.49875567,  0.61102238,  0.22123375,
        -0.03391293],
       [ 0.24573886,  0.69989466, -1.20854942,  0.61102238,  0.36767567,
         0.0736289 ],
       [-1.72305628,  0.85342053,  0.92083182,  0.61102238,  0.43782599,
         0.02193366],
       ...,
       [ 0.90200391, -1.75651931,  1.63062557,  0.61102238,  0.37550202,
         0.03075116],
       [ 0.90200391, -0.83536407, -0.49875567,  0.61102238,  0.29080635,
        -0.02926956],
       [-0.08239366,  0.69989466, -0.49875567,  0.61102238,  0.13615038,
        -0.0278006 ]])

## Usamos un clasificador con el ensemble Random Forest

In [31]:
clf = RandomForestClassifier(max_features="log2", max_depth=11, n_estimators=24,
                             min_samples_split=1000, oob_score=True)
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=11, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=1000,
            min_weight_fraction_leaf=0.0, n_estimators=24, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

## Entrenamos el modelo

In [32]:
clf.fit(x_training,y_training)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=11, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=1000,
            min_weight_fraction_leaf=0.0, n_estimators=24, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [33]:
print("Training set score: %f" % clf.score(x_training, y_training))
print("Test set score: %f" % clf.score(x_test, y_test))

Training set score: 0.362146
Test set score: 0.359434


## Predicciones y Evaluación
Ahora que tenemos un modelo, es hora de usarlo para obtener predicciones 

In [34]:
predictions = clf.predict(x_test)
predictions

array([16, 16, 16, ..., 16, 16, 16], dtype=int8)

Ahora podemos usar las métricas integradas de SciKit-Learn, como un informe de clasificación y una matriz de confusión para evaluar el rendimiento de nuestro modelo:

In [35]:
from sklearn.metrics import classification_report,confusion_matrix
m = confusion_matrix(y_test,predictions)
m.size

1521

In [36]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       339
          1       0.20      0.05      0.08     15987
          2       0.00      0.00      0.00        66
          3       0.00      0.00      0.00        68
          4       0.00      0.00      0.00      7416
          5       0.00      0.00      0.00       747
          6       0.00      0.00      0.00       435
          7       0.38      0.38      0.38      8504
          8       0.00      0.00      0.00       696
          9       0.00      0.00      0.00       225
         10       0.00      0.00      0.00        73
         11       0.00      0.00      0.00        78
         12       0.33      0.01      0.02      1642
         13       0.32      0.00      0.00      3386
         14       0.00      0.00      0.00        31
         15       0.00      0.00      0.00       374
         16       0.36      0.94      0.52     44670
         17       0.00      0.00      0.00   

## Otra forma de calcular el accuracy

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
accuracy_score(y_test, predictions)

0.35943384831614894

# Probaremos con varios modelos adicionales

In [39]:
scoring = 'accuracy'

In [42]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [45]:
results = []
names = []
seed = 7

In [ ]:
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, x_training, y_training, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.321792 (0.001520)
LDA: 0.320461 (0.001383)
KNN: 0.296550 (0.000955)
CART: 0.294909 (0.001151)
NB: 0.142842 (0.023756)


In [ ]:
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
knn = KNeighborsClassifier()
knn.fit(x_training, y_training)
predictions = knn.predict(x_test)

In [ ]:
svm = SVC()
svm.fit(x_training, y_training)
predictions = svm.predict(y_test)